In [ ]:
%load_ext autoreload
%autoreload 2

import jCMIP as jc
import os
import glob

In [ ]:
dates   = '230122'
#regions = ['Atlantic','IndoPacific','Global','BayOfBengal','ArabianSea']
regions = ['Atlantic','IndoPacific','Global']
cmips   = ['6','5']
#EXPs    = ['historical','rcp26','rcp45','rcp85','ssp126','ssp245','ssp370','ssp585','piControl']
EXPs    = ['historical','rcp26','rcp85','ssp126','ssp585']
ctypes  = ['None','Tbar','vbar']

for region in regions:
    for cmip in cmips:
        Clist = jc.readList(('../CMIP' + cmip + 'list'))
        Models = list(Clist.keys())
        for EXP in EXPs:
            for ctype in ctypes:
                names_short    = ('(')
                ens_short      = ('(')
                outfiles_short = ('(')
                runfiles_short = ('(')
                gn_short       = ('(')
                names_high     = ('(')
                ens_high       = ('(')
                outfiles_high  = ('(')
                runfiles_high  = ('(')
                gn_high        = ('(')
                ns = 0 
                nh = 0
                # Make arrays for slurm file:
                for mm in Models:
                    Model = Clist[mm]
                    print(Model.name)
                    if ((Model.Ogrid != 'Unstructured') & (os.path.isfile(Model.Omeshmask))): 
                        # Get ensemble members:
                        ENSs = Model.getENSs(var='thetao',EXP=EXP,vtype='Omon',gtype='*')
                        for ENS in ENSs:
                            files = glob.glob(Model.savedir + Model.name + '/' + EXP + '/' + ENS + '/' + region + '_' +'trans_' + ctype + '_' 
                                                   + Model.name + '_' + EXP + '_' + ENS + '_*_' + dates + '.nc')
                            if len(files) == 0:
                                gtype = 'gn'
                                # Check if files needed for computation exist:
                                Tfiles = Model.getFiles(var='thetao',EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)
                                Sfiles = Model.getFiles(var='so'    ,EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)
                                Vfiles = Model.getFiles(var='vo'    ,EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)
                                if (((len(Tfiles) == 0) | (len(Sfiles) == 0) | (len(Vfiles) == 0)) & (cmip == '6') | (Model.name[0:7] == 'NorESM2')):
                                    gtype = 'gr'
                                    Tfiles = Model.getFiles(var='thetao',EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)
                                    Sfiles = Model.getFiles(var='so'    ,EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)
                                    Vfiles = Model.getFiles(var='vo'    ,EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)
                                    if (((len(Tfiles) == 0) | (len(Sfiles) == 0) | (len(Vfiles) == 0)) & (cmip == '6')):
                                        gtype = 'gr1'
                                        Tfiles = Model.getFiles(var='thetao',EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)
                                        Sfiles = Model.getFiles(var='so'    ,EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)
                                        Vfiles = Model.getFiles(var='vo'    ,EXP=EXP,ENS=ENS,vtype='Omon',gtype=gtype)

                                if ((EXP != 'piControl') & (ctype == 'Tbar')):
                                    sfile = ('/gws/pw/j07/wishbone/jmecki/CMIP' + Model.cmip + '/' + Model.name + '/historical/' + ENS + '/so_' + Model.name + '_historical_' + ENS + '_' + gtype + '_197001-199912_SC.nc')
                                    tfile = ('/gws/pw/j07/wishbone/jmecki/CMIP' + Model.cmip + '/' + Model.name + '/historical/' + ENS + '/thetao_' + Model.name + '_historical_' + ENS + '_' + gtype + '_197001-199912_SC.nc')
                                    if ((not os.path.isfile(tfile)) | (not os.path.isfile(sfile))):
                                        Tfiles = []
                                        Sfiles = []
                                elif ((EXP != 'piControl') & (ctype == 'vbar')):
                                    vfile = ('/gws/pw/j07/wishbone/jmecki/CMIP' + Model.cmip + '/' + Model.name + '/historical/' + ENS + '/vo_' + Model.name + '_historical_' + ENS + '_' + gtype + '_197001-199912_SC.nc')
                                    if ((not os.path.isfile(vfile))):
                                        Vfiles = []


                                if ((len(Tfiles) != 0) & (len(Sfiles) != 0) & (len(Vfiles) != 0)):
                                    # Check if computation has already been completed:
                                    outfile = (Model.savedir + Model.name + '/' + EXP + '/' + ENS + '/' + region + '_' +'trans_' + ctype + '_' 
                                               + Model.name + '_' + EXP + '_' + ENS + '_' + gtype + '_' + dates + '.nc')

                                    if not os.path.isfile(outfile):
                                        # Add file to list for computing:
                                        runfile = ('/work/scratch-nopw/jmecki/transports/' + EXP + '/' + ENS + '/' + region + '_' +'trans_' + ctype + '_' 
                                                   + Model.name + '_' + EXP + '_' + ENS + '_' + gtype + '.nc')
                                        #runfile = ('/work/scratch-nopw/jmecki/transports/' + EXP + '/' + ENS + '/' + region + '_' +'trans_' + ctype + '_' 
                                        #           + Model.name + '_' + EXP + '_' + ENS + '_' + gtype + '_' + dates + '.nc')
                                        print('computing: ' + outfile)

                                        if ((Model.name == 'HadGEM3-GC31-MM') | (Model.name == 'GFDL-CM4') | (Model.name == 'CMCC-CM2-HR4') |
                                                (Model.name == 'MIROC4h') | (Model.name == 'MPI-ESM1-2-HR') | (Model.name == 'MPI-ESM-MR')):
                                            nh = nh + 1
                                            if nh == 1:
                                                names_high    = (names_high  + '"'+ Model.name + '"')
                                                ens_high      = (ens_high  + '"'+ ENS + '"')
                                                outfiles_high = (outfiles_high  + '"'+ outfile + '"')
                                                runfiles_high = (runfiles_high  + '"'+ runfile + '"')
                                                gn_high       = (gn_high  + '"'+ gtype + '"')
                                            else:
                                                names_high    = (names_high + ' "' + Model.name + '"')
                                                ens_high      = (ens_high + ' "' + ENS + '"')
                                                outfiles_high = (outfiles_high + ' "' + outfile + '"')
                                                runfiles_high = (runfiles_high + ' "' + runfile + '"')
                                                gn_high       = (gn_high + ' "' + gtype + '"')
                                        else:   
                                            ns = ns + 1
                                            if ns == 1:
                                                names_short    = (names_short  + '"' +  Model.name + '"')
                                                ens_short      = (ens_short  + '"'+  ENS + '"')
                                                outfiles_short = (outfiles_short  + '"'+  outfile + '"')
                                                runfiles_short = (runfiles_short  + '"'+  runfile + '"')
                                                gn_short       = (gn_short  + '"'+  gtype + '"')
                                            else:
                                                names_short    = (names_short + ' "' + Model.name + '"')
                                                ens_short      = (ens_short + ' "' + ENS + '"')
                                                outfiles_short = (outfiles_short + ' "' + outfile + '"')
                                                runfiles_short = (runfiles_short + ' "' + runfile + '"')
                                                gn_short       = (gn_short + ' "' + gtype + '"')  


                    # Create slurm files:
                    if ns != 0:

                        array_short = ('slurm_files_array/SC_transports_' + region + '_' + cmip + '_' + EXP + '_' + ctype + '_short.slurm')
                        line1 = '#!/bin/bash \n'
                        line2 = '#SBATCH --account=short4hr \n'
                        line3 = '#SBATCH -p short-serial-4hr \n'
                        line4 = '#SBATCH -o %J.out \n'
                        line5 = '#SBATCH -e %J.err \n'
                        line6 = '#SBATCH --time 4:00:00 \n'
                        line7 = '#SBATCH --mem=32000 \n'
                        line8 = ('#SBATCH --array=0-' + str(ns-1))
                        line9 = '\n'
                        line10= 'MODELS=' + names_short + ') \n'
                        line11= 'ENS=' + ens_short + ') \n'
                        line12= 'OUT=' + outfiles_short + ') \n'
                        line13= 'RUN=' + runfiles_short + ') \n'
                        line14= 'GN=' + gn_short + ') \n'
                        line15= 'conda activate working \n'
                        line16= ('python ../compute_transports_array.py ' + cmip + ' ${MODELS[$SLURM_ARRAY_TASK_ID]} ' + EXP + ' ${ENS[$SLURM_ARRAY_TASK_ID]} '
                                                + region + ' ${GN[$SLURM_ARRAY_TASK_ID]} ' + ctype + ' ${OUT[$SLURM_ARRAY_TASK_ID]} ${RUN[$SLURM_ARRAY_TASK_ID]}')
                        file = open(array_short,'w') 
                        file.writelines([line1, line2, line3, line4, line5, line6, line7, line8, line9, line10, line11, line12, line13, line14, line15, line16])

                        file.close()

                    if nh != 0:
                        array_high  = ('slurm_files_array/SC_transports_' + region + '_' + cmip + '_' + EXP + '_' + ctype + '_high.slurm')
                        line1 = '#!/bin/bash \n'
                        line3 = '#SBATCH -p high-mem \n'
                        line4 = '#SBATCH -o %J.out \n'
                        line5 = '#SBATCH -e %J.err \n'
                        line6 = '#SBATCH --time 12:00:00 \n'
                        line7 = '#SBATCH --mem=64000 \n'
                        line8 = ('#SBATCH --array=0-' + str(nh-1))
                        line9 = '\n'
                        line10= 'MODELS=' + names_high + ') \n'
                        line11= 'ENS=' + ens_high + ') \n'
                        line12= 'OUT=' + outfiles_high + ') \n'
                        line13= 'RUN=' + runfiles_high + ') \n'
                        line14= 'GN=' + gn_high + ') \n'
                        line15= 'conda activate working \n'
                        line16= ('python ../compute_transports_array.py ' + cmip + ' ${MODELS[$SLURM_ARRAY_TASK_ID]} ' + EXP + ' ${ENS[$SLURM_ARRAY_TASK_ID]} '
                                                + region + ' ${GN[$SLURM_ARRAY_TASK_ID]} ' + ctype + ' ${OUT[$SLURM_ARRAY_TASK_ID]} ${RUN[$SLURM_ARRAY_TASK_ID]}')
                        file = open(array_high,'w') 
                        file.writelines([line1, line3, line4, line5, line6, line7, line8, line9, line10, line11, line12, line13, line14, line15, line16])

                        file.close()